In [1]:
from glob2 import glob
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
all_file_path=glob('../artifacts/sampledb/*.edf')
print(len(all_file_path))

6


In [3]:
all_file_path[0]

'../artifacts/sampledb\\hlp41.edf'

In [4]:
healthy_file_path=[i for i in all_file_path if  'h' in i.split('\\')[1]]
patient_file_path=[i for i in all_file_path if  's' in i.split('\\')[1]]

In [5]:
def read_data(file_path):
    datax=mne.io.read_raw_edf(file_path,preload=True)
    datax.set_eeg_reference()
    datax.filter(l_freq=1,h_freq=45)
    epochs=mne.make_fixed_length_epochs(datax,duration=25,overlap=0)
    epochs=epochs.get_data()
    return epochs #trials,channel,length

In [6]:
data=read_data(healthy_file_path[0])

Extracting EDF parameters from D:\FinalYearProject\SA_Detection_CNN\artifacts\sampledb\hlp41.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5852499  =      0.000 ... 23409.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
936 matching events found
No baseline correction applied
0 project

In [7]:
data.shape


(936, 7, 6250)

In [8]:
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]

Extracting EDF parameters from D:\FinalYearProject\SA_Detection_CNN\artifacts\sampledb\hlp41.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5852499  =      0.000 ... 23409.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
936 matching events found
No baseline correction applied
0 project

In [9]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]
print(len(control_epochs_labels),len(patients_epochs_labels))

2 4


In [10]:
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
print(len(data_list),len(label_list))

6 6


In [11]:
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]

In [12]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(groups_list)
data_array=np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape,group_array.shape)

(5028, 6250, 7) (5028,) (5028,)


In [14]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/1c/c3/17c6aa1dd5bc8cea5bf00d0c3a021a5dd1680c250861cc877a7e556e4b9b/tensorflow-2.14.0-cp310-cp310-win_amd64.whl.metadata
  Using cached tensorflow-2.14.0-cp310-cp310-win_amd64.whl.metadata (3.3 kB)
  Obtaining dependency information for tensorflow-intel==2.14.0 from https://files.pythonhosted.org/packages/cb/90/599c79a248dcae6935331113649de5d75427e320efde21b583648b498584/tensorflow_intel-2.14.0-cp310-cp310-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.14.0-cp310-cp310-win_amd64.whl.metadata (4.8 kB)
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.5.26

In [15]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(6250,7)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 6248, 5)           110       
                                                                 
 batch_normalization (Batch  (None, 6248, 5)           20        
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 6248, 5)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 3124, 5)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 3122, 5)           80        
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 3122, 5)           0

In [20]:
!pip install scikit-learn


  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/59/ed/548f6f686845d386a727a51a3daa411d95fc599649a2d54705f6773ac259/scikit_learn-1.3.1-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.3 MB 1.3 MB/s eta 0:00:08
   ---------------------------------------- 0.1/9.3 MB 812.7 kB/s eta 0:00:12
   ---------------------------------------- 0.1/9.3 MB 919.0 kB/s eta 0:00:10
    --------------------------------------- 0.2/9.3 MB 1.1 

In [21]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()

In [22]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=50,batch_size=10,validation_data=(val_features,val_labels))
    accuracy.append(model.evaluate(val_features,val_labels)[1])

Epoch 1/50
410/410 [==============================] - 18s 29ms/step - loss: 0.4294 - accuracy: 0.7754 - val_loss: 0.8961 - val_accuracy: 0.0064
Epoch 2/50
410/410 [==============================] - 11s 27ms/step - loss: 0.3892 - accuracy: 0.7791 - val_loss: 0.7392 - val_accuracy: 0.3590
Epoch 3/50
410/410 [==============================] - 11s 27ms/step - loss: 0.3591 - accuracy: 0.8045 - val_loss: 0.3982 - val_accuracy: 0.9541
Epoch 4/50
410/410 [==============================] - 11s 27ms/step - loss: 0.3074 - accuracy: 0.8539 - val_loss: 0.4282 - val_accuracy: 0.8889
Epoch 5/50
410/410 [==============================] - 11s 27ms/step - loss: 0.2700 - accuracy: 0.8839 - val_loss: 1.1733 - val_accuracy: 0.5256
Epoch 6/50
410/410 [==============================] - 11s 28ms/step - loss: 0.2416 - accuracy: 0.9032 - val_loss: 1.9033 - val_accuracy: 0.3654
Epoch 7/50
410/410 [==============================] - 12s 29ms/step - loss: 0.2189 - accuracy: 0.9066 - val_loss: 0.9022 - val_accuracy:

In [23]:
np.mean(accuracy)

0.535629004612565

In [27]:
import pickle
with open('../models/cnnmodel.pickle' , 'wb')as file:
     pickle.dump(model, file)